In [ ]:
import brilearn
import utilities
import metrics
import pandas as pd
import numpy as np

Loading a pre-tuned model and making a prediction on some data

In [ ]:
model = brilearn.Model()
model.load_model()
X = pd.read_excel("prediction_data.xlsx", index_col="Index")
X = utilities.meyer_coeffs(X)
Y = model.predict(X)
Y.head()

Fitting a model on a given database, and evaluating the prediction of the model

In [ ]:
df_train = pd.read_excel(
    "fitting_data.xlsx", index_col="Index", sheet_name="Training"
)
df_val = pd.read_excel(
    "fitting_data.xlsx", index_col="Index", sheet_name="Validation"
)
df_test = pd.read_excel("fitting_data.xlsx", index_col="Index", sheet_name="Testing")

X_train = df_train[["k", "m", "Hardness"]]
Y_train = df_train[["Y", "K", "n"]]
X_val = df_val[["k", "m", "Hardness"]]
Y_val = df_val[["Y", "K", "n"]]
X_test = df_test[["k", "m", "Hardness"]]
Y_test = df_test[["Y", "K", "n"]]

In [ ]:
model = brilearn.Model()
strains = np.linspace(0.02, 0.15, 30)
model.fit(X_train, Y_train, strains, X_val, Y_val)
Y_pred = model.predict(X_test)
Y_test = utilities.construct_graphs(Y_test, Y_pred.columns)
errors = metrics.get_errors(Y_pred, Y_test)
errors.head()